In [1]:
import pandas as pd
import numpy as np
from sklearn import datasets
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import KFold, GridSearchCV
from sklearn.svm import SVC

import sys
sys.path.append("..")
from tools import write_answer

## 1. Загрузите объекты из новостного датасета 20 newsgroups, относящиеся к категориям "космос" и "атеизм".

Обратите внимание, что загрузка данных может занять несколько минут

In [7]:
newsgroups = datasets.fetch_20newsgroups(
                    subset="all", 
                    categories=["alt.atheism", "sci.space"])
X = newsgroups.data
y = newsgroups.target

## 2. Вычислите TF-IDF-признаки для всех текстов.

Обратите внимание, что в этом задании мы предлагаем вам вычислить TF-IDF по всем данным. При таком подходе получается, что признаки на обучающем множестве используют информацию из тестовой выборки — но такая ситуация вполне законна, поскольку мы не используем значения целевой переменной из теста. На практике нередко встречаются ситуации, когда признаки объектов тестовой выборки известны на момент обучения, и поэтому можно ими пользоваться при обучении алгоритма.

In [11]:
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(X)

## 3. Подберите минимальный лучший параметр C из множества [10^-5, 10^-4, ... 10^4, 10^5] для SVM с линейным ядром (kernel='linear') при помощи кросс-валидации по 5 блокам.

Укажите параметр random_state=241 и для SVM, и для KFold. В качестве меры качества используйте долю верных ответов (accuracy).

In [14]:
grid = {"C": np.power(10.0, np.arange(-5, 6))}
cv = KFold(n_splits=5, shuffle=True, random_state=241)
clf = SVC(kernel="linear", random_state=241)
gs = GridSearchCV(clf, grid, scoring="accuracy", cv=cv)
gs.fit(X, y)

C = gs.best_params_.get('C')
C

1.0

## 4. Обучите SVM по всей выборке с оптимальным параметром C, найденным на предыдущем шаге.

In [15]:
clf = SVC(C=C, kernel="linear", random_state=241)
clf.fit(X, y)

SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto_deprecated',
  kernel='linear', max_iter=-1, probability=False, random_state=241,
  shrinking=True, tol=0.001, verbose=False)

## 5. Найдите 10 слов с наибольшим абсолютным значением веса (веса хранятся в поле coef_ у svm.SVC).

Они являются ответом на это задание. Укажите эти слова через запятую или пробел, в нижнем регистре, в лексикографическом порядке.

In [31]:
# let's look at representation of weights

print(clf.coef_.indices)
print(clf.coef_.data)

clf.coef_

[11098  6775  5107 ...  3669  3601   335]
[ 0.11331532  0.05134321  0.05445196 ... -0.2372855   0.18461124
 -0.2410018 ]


<1x28382 sparse matrix of type '<class 'numpy.float64'>'
	with 18404 stored elements in Compressed Sparse Row format>

In [34]:
words = np.array(vectorizer.get_feature_names())
word_weights = pd.Series(clf.coef_.data, 
                         index=words[clf.coef_.indices], 
                         name="weight")
word_weights.index.name = "word"

top_words = word_weights.abs().sort_values(ascending=False).head(10)
top_words

word
space       2.663165
god         1.920379
atheism     1.254690
atheists    1.249180
moon        1.201611
sky         1.180132
religion    1.139081
bible       1.130612
keith       1.097094
sci         1.029307
Name: weight, dtype: float64

In [35]:
write_answer(1, " ".join(top_words.index.sort_values(ascending=True)))

atheism atheists bible god keith moon religion sci sky space
